<h1 style="color:blue;">Redes Neuronales Convolucionales para la detección de puntos anatómicos en imágenes cefálicas laterales</h1>

<p>En este proyecto busco utilizar algunas arquitecturas de redes neuronales convolucionales para determinar cual de ellas se adecúa mejor a la detección de puntos anatómicos en imágenes radiográficas cefálicas laterales, intentaré hacerlo lo mas sencillo posible.</p>

<h2 style="color:red;">Data Utilizada</h2>

<p>Para realizar esta investigación, voy a usar una data disponible de forma gratuita en <a href="http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html">Dataset</a>, de la misma, solo utilice la data con labels "senior".</p>

<h3>Breve Descripción de la data</h3>

<p>Los datos a utilizar consisten en 400 imágenes de imágenes radiográficas cefálicas laterales (escala de grises) junto con un vector que indica la posición de 19 puntos anatómicos, etiquetados por un especialista senior en el área (ver link del dataset para mayor detalle).</p>

<h2 style="color:red;">Preprocesamiento y Data Augmentation</h2>

<p>Para hacer el modelo más general, las imágenes fueron preprocesadas para cumplir con:</p>

* Imágenes cuadradas de 128x128 pixeles.
* Imágenes normalizadas (rango de intensidad de pixeles de -1 a 1).
* Para tener mayor cantidad de datos y evitar overfit, se aumentó la data al hacer flip horizontal y vertical, para un total de 1200 imágenes con sus labels (400 originales + 400 flip horizontal + 400 flip vertical).
* Igualmente, para evitar overfit, las imágenes están organizadas en 1 original - 1 flip h - 1 flip v. Nota: Es cierto que lo ideal sería mezclar todo, sin embargo, las imágenes "flipeadas" distan tanto de la original que no creo que el efecto sea tan evidente, de todas formas es un punto a explorar.
* El training set consta de 900 imágenes, mientras que el test set de 300 imágenes.


<p>Tomemos una imagen de ejemplo junto con sus labels:</p>

<img src="imagen1.png" alt="Drawing" style="width: 450px;"/>

<p>Naturalmente, la imagen no es nítida debido a que tiene solo 128x128 píxeles de resolución. Los puntos rojos representan los puntos anatómicos etiquetados por el especialista.</p>

<h2 style="color:red;">Entorno de Ejecución</h2>

<p>Debido a diversas limitantes, la mejor opción que tengo disponible es, sin dudas, usando el entorno acelerado por GPU en Google Colab (no se imaginan cuanto lo agardezco), la forma de setearlo la obtuve de este post de <a href="https://www.kdnuggets.com/2018/02/google-colab-free-gpu-tutorial-tensorflow-keras-pytorch.html">KDnuggets.</a> Recomiendo ampliamente darle una hojeada para entender mejor como funciona.<br><br>
    Para hacer mas sencillo el código, se utilizará Keras con backend TensorFlow, lo cual hace el modelo muy fácil de armar, depurar y leer. La data está almacenada en mi Google Drive, la compartiré en cuanto esté seguro que tengo permiso de hacerlo, igual la data original es accesible en el link de la sección "Data Utilizada".</p>


<h1>Estableciendo la conexión con nuestro Google Drive</h1>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

<p>El código anterior tendrá dos puntos en los que nos indica un link al cual debemos acceder para iniciar nuestra sesión en Google y darle autorización a nuestra aplicación para acceder al Drive. Es tan sencillo como copiar los códigos de autenticación en los campos indicados.</p>

<p>Procedemos a montar nuestro Drive a través de la referencia "drive"</p>

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

<p>Ahora, verificamos que nuestro entorno en Google Colab tiene configurado el entorno acelerado por GPU, lo cual nos debería arrojar <b>['/device:GPU:0']</b> en caso afirmativo.</p>

<p><b>Nota: </b> El programa debe funcionar aun cuando no se encuentre acelerado por GPU, sin embargo, la diferencia en rapidez (y por tanto en viabilidad de la investigación) es muy considerable en mi caso, siendo alrededor de 20 a 30 veces más rápido, sin contar el hecho de que el computador no queda "esclavizado" por el uso exhaustivo de CPU y Memoria RAM.</p>

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

<h1>Importando librerías necesarias y cargando el training y test set</h1>

In [ ]:
!pip install -q keras #En caso de que no lo esté

import pickle
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, concatenate
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras.optimizers
from keras.utils import plot_model
import h5py

<p>Cargamos la data desde nuestro Drive en un diccionario con dos índices princiaples:</p>

* Diccionario['Img'] para obtener las imágenes (matriz de intensidad de grises).
* Diccionario['Lbl'] para obtener los labels.

<p>Es importante notar que índices iguales corresponden al par Imagen-Label respectivo, esto es, Diccionario['Img'][15] es una imagen cuyo label está guardado en Diccionario['Lbl'][15].</p>
    
<p>La ruta dentro de mi sesión de Google Drive para obtener la data a evaluar es "drive/ColabRuns/ImagenesFlipLabelsSenior128.pickle". Como se puede notar, es un archivo tipo 'pickle', para facilitar su lectura y por ser ligero en espacio que ocupa.</p>

In [ ]:
Diccionario = pickle.load(open("drive/ColabRuns/ImagenesFlipLabelsSenior128.pickle","rb"))

<p>Definimos nuestras variables de training y test según lo especificado anteriormente<p>

In [ ]:
X_train=np.array(Diccionario['Img'][0:900]).reshape(900,128,128,1).astype('float32')
Y_train=np.array(Diccionario['Lbl'][0:900]).reshape(900,38).astype('float32')

X_test=np.array(Diccionario['Img'][900:1200]).reshape(300,128,128,1).astype('float32')
Y_test=np.array(Diccionario['Lbl'][900:1200]).reshape(300,38).astype('float32')

<p>Algunas notas sobre el reshape y el astype:</p>
* reshape(900,128,128,1) corresponde a 900 imágenes, de 128 x 128 pixeles, de 1 canal cada una.
* reshape(900,38) corresponde a 900 labels, cada uno de 38 elementos (19 pares de coordenadas).
* astype('float32') es un tipo de dato aceptado por TensorFlow y Keras, cuando se guardó la data en un archivo 'pickle', los datos pueden no ser devueltos en un formato aceptable.

<p>Una vez cumplido lo explicado anteriormente, llegamos al punto en que debemos definir la arquitectura de red neuronal convolucional en si, en este caso, usaremos una arquitectura del tipo "AlexNet", que no es mas que una arquitectura secuencial, pero que se reconoce como el punto de inflexión que hizo posible el actual desarrollo y uso de las CNNs en el área de visión por computador.</p>

<p><b>Nota: </b>Las demás arquitecturas a evaluar serán exactamente iguales hasta aquí.</p>

<h1>Definición del Modelo: Inception</h1>

<p>El modelo está basado en el trabajo de Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed, Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich:</p>
<br><b>
Going Deeper with Convolutions. Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed, Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich
arXiv:1409.4842
</b><br>

In [ ]:
input1 = Input(shape=(128,128,1))

Conv1 = Convolution2D(64, kernel_size=5, strides=3, activation='relu', padding='valid')(input1)
Maxp1 = MaxPooling2D(pool_size=(3,3), strides=2, padding='valid')(Conv1)
Conv2 = Convolution2D(192, kernel_size=3, strides=1, activation='relu', padding='same')(Maxp1)
Maxp2 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(Conv2)

#Stack 1
tower_0 = Convolution2D(64, (1, 1), padding='same', activation='relu')(Maxp2)

tower_1 = Convolution2D(96, (1, 1), padding='same', activation='relu')(Maxp2)
tower_1 = Convolution2D(128, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Convolution2D(16, (1, 1), padding='same', activation='relu')(Maxp2)
tower_2 = Convolution2D(32, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(Maxp2)
tower_3 = Convolution2D(32, (1, 1), padding='same', activation='relu')(tower_3)

output1 = keras.layers.concatenate([tower_0, tower_1, tower_2, tower_3], axis=3)

#Stack 2
tower_4 = Convolution2D(128, (1, 1), padding='same', activation='relu')(output1)

tower_5 = Convolution2D(128, (1, 1), padding='same', activation='relu')(output1)
tower_5 = Convolution2D(192, (3, 3), padding='same', activation='relu')(tower_5)

tower_6 = Convolution2D(32, (1, 1), padding='same', activation='relu')(output1)
tower_6 = Convolution2D(96, (5, 5), padding='same', activation='relu')(tower_6)

tower_7 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output1)
tower_7 = Convolution2D(64, (1, 1), padding='same', activation='relu')(tower_7)

output2 = keras.layers.concatenate([tower_4, tower_5, tower_6, tower_7], axis=3)



Maxp3 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(output2)


#Stack 3
tower_8 = Convolution2D(192, (1, 1), padding='same', activation='relu')(output2)

tower_9 = Convolution2D(96, (1, 1), padding='same', activation='relu')(output2)
tower_9 = Convolution2D(208, (3, 3), padding='same', activation='relu')(tower_9)

tower_10 = Convolution2D(16, (1, 1), padding='same', activation='relu')(output2)
tower_10 = Convolution2D(48, (5, 5), padding='same', activation='relu')(tower_10)

tower_11 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output2)
tower_11 = Convolution2D(64, (1, 1), padding='same', activation='relu')(tower_11)

output3 = keras.layers.concatenate([tower_8, tower_9, tower_10, tower_11], axis=3)


#Stack 4
tower_12 = Convolution2D(160, (1, 1), padding='same', activation='relu')(output3)

tower_13 = Convolution2D(112, (1, 1), padding='same', activation='relu')(output3)
tower_13 = Convolution2D(224, (3, 3), padding='same', activation='relu')(tower_13)

tower_14 = Convolution2D(24, (1, 1), padding='same', activation='relu')(output3)
tower_14 = Convolution2D(64, (5, 5), padding='same', activation='relu')(tower_14)

tower_15 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output3)
tower_15 = Convolution2D(64, (1, 1), padding='same', activation='relu')(tower_15)

output4 = keras.layers.concatenate([tower_12, tower_13, tower_14, tower_15], axis=3)


#Stack 5
tower_16 = Convolution2D(128, (1, 1), padding='same', activation='relu')(output4)

tower_17 = Convolution2D(128, (1, 1), padding='same', activation='relu')(output4)
tower_17 = Convolution2D(256, (3, 3), padding='same', activation='relu')(tower_17)

tower_18 = Convolution2D(24, (1, 1), padding='same', activation='relu')(output4)
tower_18 = Convolution2D(64, (5, 5), padding='same', activation='relu')(tower_18)

tower_19 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output4)
tower_19 = Convolution2D(64, (1, 1), padding='same', activation='relu')(tower_19)

output5 = keras.layers.concatenate([tower_16, tower_17, tower_18, tower_19], axis=3)


#Stack 6
tower_20 = Convolution2D(112, (1, 1), padding='same', activation='relu')(output5)

tower_21 = Convolution2D(144, (1, 1), padding='same', activation='relu')(output5)
tower_21 = Convolution2D(288, (3, 3), padding='same', activation='relu')(tower_21)

tower_22 = Convolution2D(32, (1, 1), padding='same', activation='relu')(output5)
tower_22 = Convolution2D(64, (5, 5), padding='same', activation='relu')(tower_22)

tower_23 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output5)
tower_23 = Convolution2D(64, (1, 1), padding='same', activation='relu')(tower_23)

output6 = keras.layers.concatenate([tower_20, tower_21, tower_22, tower_23], axis=3)


#Stack 7
tower_24 = Convolution2D(256, (1, 1), padding='same', activation='relu')(output6)

tower_25 = Convolution2D(160, (1, 1), padding='same', activation='relu')(output6)
tower_25 = Convolution2D(320, (3, 3), padding='same', activation='relu')(tower_25)

tower_26 = Convolution2D(32, (1, 1), padding='same', activation='relu')(output6)
tower_26 = Convolution2D(128, (5, 5), padding='same', activation='relu')(tower_26)

tower_27 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output6)
tower_27 = Convolution2D(128, (1, 1), padding='same', activation='relu')(tower_27)

output7 = keras.layers.concatenate([tower_24, tower_25, tower_26, tower_27], axis=3)


Maxp4 = MaxPooling2D(pool_size=(3,3), strides=2)(output7)


#Stack 8
tower_28 = Convolution2D(256, (1, 1), padding='same', activation='relu')(Maxp4)

tower_29 = Convolution2D(160, (1, 1), padding='same', activation='relu')(Maxp4)
tower_29 = Convolution2D(320, (3, 3), padding='same', activation='relu')(tower_29)

tower_30 = Convolution2D(32, (1, 1), padding='same', activation='relu')(Maxp4)
tower_30 = Convolution2D(128, (5, 5), padding='same', activation='relu')(tower_30)

tower_31 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(Maxp4)
tower_31 = Convolution2D(128, (1, 1), padding='same', activation='relu')(tower_31)

output8 = keras.layers.concatenate([tower_28, tower_29, tower_30, tower_31], axis=3)


#Stack 9
tower_32 = Convolution2D(384, (1, 1), padding='same', activation='relu')(output8)

tower_33 = Convolution2D(192, (1, 1), padding='same', activation='relu')(output8)
tower_33 = Convolution2D(384, (3, 3), padding='same', activation='relu')(tower_33)

tower_34 = Convolution2D(48, (1, 1), padding='same', activation='relu')(output8)
tower_34 = Convolution2D(128, (5, 5), padding='same', activation='relu')(tower_34)

tower_35 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(output8)
tower_35 = Convolution2D(128, (1, 1), padding='same', activation='relu')(tower_35)

output9 = keras.layers.concatenate([tower_32, tower_33, tower_34, tower_35], axis=3)

Avg1 = AveragePooling2D(pool_size=(4,4), strides=1)(output9)
DO1 = Dropout(0.4)(Avg1)
flat = Flatten()(DO1)
FC1 = Dense(38, activation='relu')(flat)

model = Model(inputs=input1, outputs=FC1)

<p>Veamos algunas características del modelo:<p>

<h3>Resumen:</h3>


<pre>
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_49 (InputLayer)           (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_879 (Conv2D)             (None, 42, 42, 64)   1664        input_49[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_259 (MaxPooling2D (None, 20, 20, 64)   0           conv2d_879[0][0]                 
__________________________________________________________________________________________________
conv2d_880 (Conv2D)             (None, 20, 20, 192)  110784      max_pooling2d_259[0][0]          
__________________________________________________________________________________________________
max_pooling2d_260 (MaxPooling2D (None, 10, 10, 192)  0           conv2d_880[0][0]                 
__________________________________________________________________________________________________
conv2d_882 (Conv2D)             (None, 10, 10, 96)   18528       max_pooling2d_260[0][0]          
__________________________________________________________________________________________________
conv2d_884 (Conv2D)             (None, 10, 10, 16)   3088        max_pooling2d_260[0][0]          
__________________________________________________________________________________________________
max_pooling2d_261 (MaxPooling2D (None, 10, 10, 192)  0           max_pooling2d_260[0][0]          
__________________________________________________________________________________________________
conv2d_881 (Conv2D)             (None, 10, 10, 64)   12352       max_pooling2d_260[0][0]          
__________________________________________________________________________________________________
conv2d_883 (Conv2D)             (None, 10, 10, 128)  110720      conv2d_882[0][0]                 
__________________________________________________________________________________________________
conv2d_885 (Conv2D)             (None, 10, 10, 32)   12832       conv2d_884[0][0]                 
__________________________________________________________________________________________________
conv2d_886 (Conv2D)             (None, 10, 10, 32)   6176        max_pooling2d_261[0][0]          
__________________________________________________________________________________________________
concatenate_87 (Concatenate)    (None, 10, 10, 256)  0           conv2d_881[0][0]                 
                                                                 conv2d_883[0][0]                 
                                                                 conv2d_885[0][0]                 
                                                                 conv2d_886[0][0]                 
__________________________________________________________________________________________________
conv2d_888 (Conv2D)             (None, 10, 10, 128)  32896       concatenate_87[0][0]             
__________________________________________________________________________________________________
conv2d_890 (Conv2D)             (None, 10, 10, 32)   8224        concatenate_87[0][0]             
__________________________________________________________________________________________________
max_pooling2d_262 (MaxPooling2D (None, 10, 10, 256)  0           concatenate_87[0][0]             
__________________________________________________________________________________________________
conv2d_887 (Conv2D)             (None, 10, 10, 128)  32896       concatenate_87[0][0]             
__________________________________________________________________________________________________
conv2d_889 (Conv2D)             (None, 10, 10, 192)  221376      conv2d_888[0][0]                 
__________________________________________________________________________________________________
conv2d_891 (Conv2D)             (None, 10, 10, 96)   76896       conv2d_890[0][0]                 
__________________________________________________________________________________________________
conv2d_892 (Conv2D)             (None, 10, 10, 64)   16448       max_pooling2d_262[0][0]          
__________________________________________________________________________________________________
concatenate_88 (Concatenate)    (None, 10, 10, 480)  0           conv2d_887[0][0]                 
                                                                 conv2d_889[0][0]                 
                                                                 conv2d_891[0][0]                 
                                                                 conv2d_892[0][0]                 
__________________________________________________________________________________________________
conv2d_894 (Conv2D)             (None, 10, 10, 96)   46176       concatenate_88[0][0]             
__________________________________________________________________________________________________
conv2d_896 (Conv2D)             (None, 10, 10, 16)   7696        concatenate_88[0][0]             
__________________________________________________________________________________________________
max_pooling2d_264 (MaxPooling2D (None, 10, 10, 480)  0           concatenate_88[0][0]             
__________________________________________________________________________________________________
conv2d_893 (Conv2D)             (None, 10, 10, 192)  92352       concatenate_88[0][0]             
__________________________________________________________________________________________________
conv2d_895 (Conv2D)             (None, 10, 10, 208)  179920      conv2d_894[0][0]                 
__________________________________________________________________________________________________
conv2d_897 (Conv2D)             (None, 10, 10, 48)   19248       conv2d_896[0][0]                 
__________________________________________________________________________________________________
conv2d_898 (Conv2D)             (None, 10, 10, 64)   30784       max_pooling2d_264[0][0]          
__________________________________________________________________________________________________
concatenate_89 (Concatenate)    (None, 10, 10, 512)  0           conv2d_893[0][0]                 
                                                                 conv2d_895[0][0]                 
                                                                 conv2d_897[0][0]                 
                                                                 conv2d_898[0][0]                 
__________________________________________________________________________________________________
conv2d_900 (Conv2D)             (None, 10, 10, 112)  57456       concatenate_89[0][0]             
__________________________________________________________________________________________________
conv2d_902 (Conv2D)             (None, 10, 10, 24)   12312       concatenate_89[0][0]             
__________________________________________________________________________________________________
max_pooling2d_265 (MaxPooling2D (None, 10, 10, 512)  0           concatenate_89[0][0]             
__________________________________________________________________________________________________
conv2d_899 (Conv2D)             (None, 10, 10, 160)  82080       concatenate_89[0][0]             
__________________________________________________________________________________________________
conv2d_901 (Conv2D)             (None, 10, 10, 224)  226016      conv2d_900[0][0]                 
__________________________________________________________________________________________________
conv2d_903 (Conv2D)             (None, 10, 10, 64)   38464       conv2d_902[0][0]                 
__________________________________________________________________________________________________
conv2d_904 (Conv2D)             (None, 10, 10, 64)   32832       max_pooling2d_265[0][0]          
__________________________________________________________________________________________________
concatenate_90 (Concatenate)    (None, 10, 10, 512)  0           conv2d_899[0][0]                 
                                                                 conv2d_901[0][0]                 
                                                                 conv2d_903[0][0]                 
                                                                 conv2d_904[0][0]                 
__________________________________________________________________________________________________
conv2d_906 (Conv2D)             (None, 10, 10, 128)  65664       concatenate_90[0][0]             
__________________________________________________________________________________________________
conv2d_908 (Conv2D)             (None, 10, 10, 24)   12312       concatenate_90[0][0]             
__________________________________________________________________________________________________
max_pooling2d_266 (MaxPooling2D (None, 10, 10, 512)  0           concatenate_90[0][0]             
__________________________________________________________________________________________________
conv2d_905 (Conv2D)             (None, 10, 10, 128)  65664       concatenate_90[0][0]             
__________________________________________________________________________________________________
conv2d_907 (Conv2D)             (None, 10, 10, 256)  295168      conv2d_906[0][0]                 
__________________________________________________________________________________________________
conv2d_909 (Conv2D)             (None, 10, 10, 64)   38464       conv2d_908[0][0]                 
__________________________________________________________________________________________________
conv2d_910 (Conv2D)             (None, 10, 10, 64)   32832       max_pooling2d_266[0][0]          
__________________________________________________________________________________________________
concatenate_91 (Concatenate)    (None, 10, 10, 512)  0           conv2d_905[0][0]                 
                                                                 conv2d_907[0][0]                 
                                                                 conv2d_909[0][0]                 
                                                                 conv2d_910[0][0]                 
__________________________________________________________________________________________________
conv2d_912 (Conv2D)             (None, 10, 10, 144)  73872       concatenate_91[0][0]             
__________________________________________________________________________________________________
conv2d_914 (Conv2D)             (None, 10, 10, 32)   16416       concatenate_91[0][0]             
__________________________________________________________________________________________________
max_pooling2d_267 (MaxPooling2D (None, 10, 10, 512)  0           concatenate_91[0][0]             
__________________________________________________________________________________________________
conv2d_911 (Conv2D)             (None, 10, 10, 112)  57456       concatenate_91[0][0]             
__________________________________________________________________________________________________
conv2d_913 (Conv2D)             (None, 10, 10, 288)  373536      conv2d_912[0][0]                 
__________________________________________________________________________________________________
conv2d_915 (Conv2D)             (None, 10, 10, 64)   51264       conv2d_914[0][0]                 
__________________________________________________________________________________________________
conv2d_916 (Conv2D)             (None, 10, 10, 64)   32832       max_pooling2d_267[0][0]          
__________________________________________________________________________________________________
concatenate_92 (Concatenate)    (None, 10, 10, 528)  0           conv2d_911[0][0]                 
                                                                 conv2d_913[0][0]                 
                                                                 conv2d_915[0][0]                 
                                                                 conv2d_916[0][0]                 
__________________________________________________________________________________________________
conv2d_918 (Conv2D)             (None, 10, 10, 160)  84640       concatenate_92[0][0]             
__________________________________________________________________________________________________
conv2d_920 (Conv2D)             (None, 10, 10, 32)   16928       concatenate_92[0][0]             
__________________________________________________________________________________________________
max_pooling2d_268 (MaxPooling2D (None, 10, 10, 528)  0           concatenate_92[0][0]             
__________________________________________________________________________________________________
conv2d_917 (Conv2D)             (None, 10, 10, 256)  135424      concatenate_92[0][0]             
__________________________________________________________________________________________________
conv2d_919 (Conv2D)             (None, 10, 10, 320)  461120      conv2d_918[0][0]                 
__________________________________________________________________________________________________
conv2d_921 (Conv2D)             (None, 10, 10, 128)  102528      conv2d_920[0][0]                 
__________________________________________________________________________________________________
conv2d_922 (Conv2D)             (None, 10, 10, 128)  67712       max_pooling2d_268[0][0]          
__________________________________________________________________________________________________
concatenate_93 (Concatenate)    (None, 10, 10, 832)  0           conv2d_917[0][0]                 
                                                                 conv2d_919[0][0]                 
                                                                 conv2d_921[0][0]                 
                                                                 conv2d_922[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_269 (MaxPooling2D (None, 4, 4, 832)    0           concatenate_93[0][0]             
__________________________________________________________________________________________________
conv2d_924 (Conv2D)             (None, 4, 4, 160)    133280      max_pooling2d_269[0][0]          
__________________________________________________________________________________________________
conv2d_926 (Conv2D)             (None, 4, 4, 32)     26656       max_pooling2d_269[0][0]          
__________________________________________________________________________________________________
max_pooling2d_270 (MaxPooling2D (None, 4, 4, 832)    0           max_pooling2d_269[0][0]          
__________________________________________________________________________________________________
conv2d_923 (Conv2D)             (None, 4, 4, 256)    213248      max_pooling2d_269[0][0]          
__________________________________________________________________________________________________
conv2d_925 (Conv2D)             (None, 4, 4, 320)    461120      conv2d_924[0][0]                 
__________________________________________________________________________________________________
conv2d_927 (Conv2D)             (None, 4, 4, 128)    102528      conv2d_926[0][0]                 
__________________________________________________________________________________________________
conv2d_928 (Conv2D)             (None, 4, 4, 128)    106624      max_pooling2d_270[0][0]          
__________________________________________________________________________________________________
concatenate_94 (Concatenate)    (None, 4, 4, 832)    0           conv2d_923[0][0]                 
                                                                 conv2d_925[0][0]                 
                                                                 conv2d_927[0][0]                 
                                                                 conv2d_928[0][0]                 
__________________________________________________________________________________________________
conv2d_930 (Conv2D)             (None, 4, 4, 192)    159936      concatenate_94[0][0]             
__________________________________________________________________________________________________
conv2d_932 (Conv2D)             (None, 4, 4, 48)     39984       concatenate_94[0][0]             
__________________________________________________________________________________________________
max_pooling2d_271 (MaxPooling2D (None, 4, 4, 832)    0           concatenate_94[0][0]             
__________________________________________________________________________________________________
conv2d_929 (Conv2D)             (None, 4, 4, 384)    319872      concatenate_94[0][0]             
__________________________________________________________________________________________________
conv2d_931 (Conv2D)             (None, 4, 4, 384)    663936      conv2d_930[0][0]                 
__________________________________________________________________________________________________
conv2d_933 (Conv2D)             (None, 4, 4, 128)    153728      conv2d_932[0][0]                 
__________________________________________________________________________________________________
conv2d_934 (Conv2D)             (None, 4, 4, 128)    106624      max_pooling2d_271[0][0]          
__________________________________________________________________________________________________
concatenate_95 (Concatenate)    (None, 4, 4, 1024)   0           conv2d_929[0][0]                 
                                                                 conv2d_931[0][0]                 
                                                                 conv2d_933[0][0]                 
                                                                 conv2d_934[0][0]                 
__________________________________________________________________________________________________
average_pooling2d_5 (AveragePoo (None, 1, 1, 1024)   0           concatenate_95[0][0]             
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 1, 1, 1024)   0           average_pooling2d_5[0][0]        
__________________________________________________________________________________________________
flatten_15 (Flatten)            (None, 1024)         0           dropout_15[0][0]                 
__________________________________________________________________________________________________
dense_71 (Dense)                (None, 38)           38950       flatten_15[0][0]                 
==================================================================================================
Total params: 6,000,534
Trainable params: 6,000,534
Non-trainable params: 0
__________________________________________________________________________________________________
None
</pre>

<h3>Grafo:</h3>

<img src="Inception1.png" alt="Drawing" style="width: 450px;"/>





<p>Definamos ahora el optimizador, condiciones de parada (stopper) y métricas de corrida:</p>

In [ ]:
optimizer=keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#Callbacks: csv_logger: Registro de la corrida. stopper: Condiciones de parada antes de las 500 iteraciones.
csv_logger = keras.callbacks.CSVLogger('drive/training_Modelo3_Vanilla_Inception.log')
stopper = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-08, patience=15, verbose=0, mode='auto')
model.compile(loss='mean_squared_error',optimizer=optimizer, metrics=['accuracy'])

<p>Ejecutemos el modelo:</p>

In [ ]:
model.fit(X_train, Y_train, batch_size=50, epochs=500, verbose=1,callbacks=[csv_logger,stopper])

<h2>Evaluación del Test Set y visualización de resultados</h2>

<p>Después de finalizada la corrida, procedemos a evaluar el modelo con el test set:</p>

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Loss del Test Set: "+str(score[0]))

<p>Loss del Test Set: 0.0030989068653434516 </p>

<p>El modelo debe ser corrido varias veces para obtener un promedio de la pérdida real, aquí solo se muestra una vez.</p>

In [ ]:
#Función para visualizar una imágen con los labels reales (rojo) vs los predichos (azules)
def probador(start=1000):
    vector = model.predict(np.array(Diccionario['Img'][start:start+1]).reshape(1,128,128,1).astype('float32'),batch_size=1,verbose=0)
    plt.imshow(np.reshape(Diccionario['Img'][start:start+1][0],[128,128]),cmap='gray')
    Y_Predicted = np.array(vector,dtype='float32').reshape([19,2])
    return plt.scatter(x=Y_Predicted[:,0]*128,y=Y_Predicted[:,1]*128, c='b', s=4),plt.scatter(x=Diccionario['Lbl'][start:start+1][0].reshape([19,2])[:,0]*128,y=Diccionario['Lbl'][start:start+1][0].reshape([19,2])[:,1]*128, c='r', s=4)

<p>Algunas imágenes de resultado, recordar que los puntos rojos son los labels "reales" y los azules los labels "predichos":</p>



<img src="ImagenesInception/1014.png" style="width: 300px; float: left;"/>

<img src="ImagenesInception/1019.png" style="width: 300px; float: left;"/>

<img src="ImagenesInception/1036.png" style="width: 300px; float: left;"/>

<h4>Comentarios:</h4>

<p>El modelo presenta un comportamiento mixto, parece realizar buen trabajo en unas imágenes y en otras no, sin embargo, en mi opinión, es el más prometedor, ya que introduce muchas layers sin un aumento considerable en la cantidad de parámetros, y que, de hecho, el modelo terminó puesto que se cumplieron las 500 iteraciones, caso que no sucedió con los modelos AlexNet y VGG19, los cuales terminaron porque no había mejoras según lo establecido en el "stopper".</p>